In [1]:
import time
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
username = 'root'
password = '1234'
host = 'localhost'
port = '3306'
database = 'discogs'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

In [11]:
def get_df(select_code):
    try:
        start_time = time.time()
        df = pd.read_sql(select_code, con=engine)
        end_time = time.time()
        
        execution_time = end_time - start_time
        print(f"Время выполнения запроса: {execution_time:.4f} секунд")
    except Exception as e:
        print(f"Ошибка: {e}") 
    return df

In [12]:
def get_query_db(query):
    try:
        with engine.connect() as connection:
            start_time = time.time()
            result = connection.execute(query)
            end_time = time.time()
            
            execution_time = end_time - start_time
            print(f"Время выполнения запроса: {execution_time:.4f} секунд")
    except Exception as e:
        print(f"Ошибка: {e}")

--------------------------------------------------

### Индексы 1
Найти информацию по заданному исполнителю, используя его имя.

In [18]:
query = text("""show indexes from artist;""")  
df = get_df(query)
df

Время выполнения запроса: 0.0080 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_id,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [19]:
# запрос без индекса

query = text("""select * from artist 
                where name = 'K.A.B.'""")  
get_df(query)

Время выполнения запроса: 21.0082 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,6,K.A.B.,Karl Axel Bissler,,Needs Major Changes


In [20]:
query = text("""CREATE INDEX idx_artist_name ON artist (name);""")
df = get_query_db(query)
df

Время выполнения запроса: 66.3796 секунд


In [22]:
# запрос с индесом
query = text("""select * from artist 
                where name = 'K.A.B.'""")  
get_df(query)

Время выполнения запроса: 0.0702 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,6,K.A.B.,Karl Axel Bissler,,Needs Major Changes


In [17]:
query = text("""drop INDEX idx_artist_name ON artist""")
df = get_query_db(query)
df

Время выполнения запроса: 0.0866 секунд


-------------------


### Индексы 2
Найти всех участников указанного музыкального коллектива (по названию коллектива).

In [23]:
query_index = text("""show indexes from `group`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0175 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,group,0,PRIMARY,1,GROUP_ID,A,1285836,None,None,,BTREE,,,YES,None


In [24]:
query_index = text("""show indexes from `artist`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0078 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_id,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,idx_artist_name,1,NAME,A,6263415,None,None,,BTREE,,,YES,None
3,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [27]:
query_2 = text("""SELECT * 
                FROM artist 
                JOIN `group` ON artist.artist_id = `group`.group_artist_id
                WHERE `group`.main_artist_id = (SELECT artist_id FROM artist WHERE name = 'DATacide');""")  

get_df(query_2)

Время выполнения запроса: 0.5454 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY,GROUP_ID,MAIN_ARTIST_ID,GROUP_ARTIST_ID
0,25,Tetsu Inoue,Tetsu Inoue,,Needs Vote,47,22,25
1,519207,Uwe Schmidt,Uwe H. Schmidt,"Born August 27th, 1968 in Frankfurt am Main, G...",Correct,263452,22,519207


In [28]:
query_index_1 = text("""CREATE INDEX idx_group_main_artist_id ON discogs.group (main_artist_id);""")
query_index_2 = text("""CREATE INDEX idx_group_artist_id ON discogs.group (group_artist_id);""")

get_query_db(query_index_1)
get_query_db(query_index_2)

Время выполнения запроса: 4.9488 секунд
Время выполнения запроса: 4.6118 секунд


In [29]:
get_df(query_2)

Время выполнения запроса: 0.0049 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY,GROUP_ID,MAIN_ARTIST_ID,GROUP_ARTIST_ID
0,25,Tetsu Inoue,Tetsu Inoue,,Needs Vote,47,22,25
1,519207,Uwe Schmidt,Uwe H. Schmidt,"Born August 27th, 1968 in Frankfurt am Main, G...",Correct,263452,22,519207


In [26]:
# удаляем индексы

query = text("""drop INDEX idx_group_main_artist_id ON `group`""")
df = get_query_db(query)

query = text("""drop INDEX idx_group_artist_id ON `group`""")
df = get_query_db(query)

Ошибка: (pymysql.err.OperationalError) (1091, "Can't DROP 'idx_group_main_artist_id'; check that column/key exists")
[SQL: drop INDEX idx_group_main_artist_id ON `group`]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Ошибка: (pymysql.err.OperationalError) (1091, "Can't DROP 'idx_group_artist_id'; check that column/key exists")
[SQL: drop INDEX idx_group_artist_id ON `group`]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


------------------------------------

### Индексы 3
Найти все релизы заданного исполнителя и отсортировать их по дате выпуска. Вывести имя исполнителя, название релиза, дату выхода.

In [30]:
query_index = text("""show indexes from `release`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0108 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release,0,PRIMARY,1,RELEASE_ID,A,10657532,None,None,,BTREE,,,YES,None
1,release,1,idx_release_main_release,1,IS_MAIN_RELEASE,A,1,None,None,,BTREE,,,YES,None
2,release,1,idx_release_year,1,RELEASED,A,29808,None,None,,BTREE,,,YES,None


In [31]:
query_index = text("""show indexes from `release_artist`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0156 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release_artist,0,PRIMARY,1,RELEASE_ARTIST_ID,A,52563772,None,None,,BTREE,,,YES,None


In [60]:
# до индексов 
query_indx_3 = text("""Explain SELECT artist.name, r.title, r.released 
                        FROM discogs.release r
                        JOIN release_artist ON r.release_id = release_artist.release_id
                        JOIN artist ON artist.artist_id = release_artist.artist_id
                        WHERE artist.name = 'K.A.B.'
                        ORDER BY r.released;""")  

get_df(query_indx_3)

Время выполнения запроса: 0.0344 секунд


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,artist,None,ref,"PRIMARY,idx_artist_id,idx_artist_name",idx_artist_name,1022,const,1,100.0,Using index; Using temporary; Using filesort
1,1,SIMPLE,release_artist,None,ALL,idx_release_id,None,None,None,52563772,10.0,Using where; Using join buffer (hash join)
2,1,SIMPLE,r,None,eq_ref,PRIMARY,PRIMARY,4,discogs.release_artist.RELEASE_ID,1,100.0,None


In [ ]:
query_1 = text("""CREATE INDEX idx_release_artist_id ON discogs.release_artist (artist_id);""")
query_2 = text("""CREATE INDEX idx_release_released ON discogs.release (released);""")
#query_3 = text("""CREATE INDEX idx_release_released ON discogs.release (released);""") (name)

get_query_db(query_1)
get_query_db(query_2)

Время выполнения запроса: 154.5388 секунд
Время выполнения запроса: 38.8230 секунд


In [34]:
# после создания индексов
get_df(query_indx_3)

Время выполнения запроса: 0.0839 секунд


,name,title,released
0,K.A.B.,Excursion,2012-10-29


In [35]:
# удаляем индексы

query = text("""drop INDEX idx_release_artist_id ON `release_artist`""")
df = get_query_db(query)

query = text("""drop INDEX idx_release_released ON `release`""")
df = get_query_db(query)

Время выполнения запроса: 0.1166 секунд
Время выполнения запроса: 0.0223 секунд


-------------------------------


### Индексы 4
Найти все главные релизы, выпущенные в указанный год, с указанием стиля релиза. Релиз является главным, если поле release.IS_MAIN_RELEASE = 1.

In [36]:
query_index = text("""show indexes from `release`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0681 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release,0,PRIMARY,1,RELEASE_ID,A,10657532,None,None,,BTREE,,,YES,None
1,release,1,idx_release_main_release,1,IS_MAIN_RELEASE,A,1,None,None,,BTREE,,,YES,None
2,release,1,idx_release_year,1,RELEASED,A,29808,None,None,,BTREE,,,YES,None


In [37]:
query_index = text("""show indexes from `style`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0112 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,style,0,PRIMARY,1,STYLE_ID,A,17868513,None,None,,BTREE,,,YES,None
1,style,1,idx_release_style,1,RELEASE_ID,A,9649605,None,None,YES,BTREE,,,YES,None


In [ ]:
query_indx_4 = text("""SELECT r.title, style.style_name
                        FROM discogs.release r
                        JOIN style ON r.release_id = style.release_id
                        WHERE r.is_main_release = 1 
                        AND YEAR(r.released) = 2023
                        AND STYLE_NAME = 'Deep House';""")  

get_query_db(query_indx_4)

Время выполнения запроса: 39.4227 секунд


In [41]:
query_1 = text("""CREATE INDEX idx_release_main_release ON discogs.release (is_main_release);""")
query_2 = text("""CREATE INDEX idx_release_year ON discogs.release (released);""")
query_3 = text("""CREATE INDEX idx_release_style ON discogs.style (release_id);""")

get_query_db(query_1)
get_query_db(query_2)
get_query_db(query_3)

Время выполнения запроса: 37.8715 секунд
Время выполнения запроса: 39.3121 секунд
Время выполнения запроса: 52.9502 секунд


In [42]:
get_query_db(query_indx_4)

Время выполнения запроса: 35.3612 секунд


In [43]:
query = text("""drop INDEX idx_release_main_release ON `release`""")
df = get_query_db(query)

query = text("""drop INDEX idx_release_year ON `release`""")
df = get_query_db(query)

query = text("""drop INDEX idx_release_style ON `style`""")
df = get_query_db(query)

Время выполнения запроса: 0.0519 секунд
Время выполнения запроса: 0.0251 секунд
Время выполнения запроса: 0.0320 секунд


------------------------------------------

### Индексы 5
Найти всех исполнителей, в описании (профиле) которых встречается указанное выражение, с использованием полнотекстового запроса.

In [58]:
query_index = text("""show indexes from `artist`;""")  
df = get_df(query_index)
df

Время выполнения запроса: 0.0101 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_id,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,idx_artist_name,1,NAME,A,6263415,None,None,,BTREE,,,YES,None
3,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [49]:
query_indx_5= text("""
                    SELECT *
                    FROM discogs.artist
                    WHERE PROFILE LIKE '%рок%';""") 
get_query_db(query_indx_5)

Время выполнения запроса: 19.5386 секунд


In [55]:
index_5 = """ALTER TABLE discogs.artist ADD FULLTEXT INDEX ft_profile (PROFILE);"""

get_query_db(index_5)

Ошибка: Not an executable object: 'ALTER TABLE discogs.artist ADD FULLTEXT INDEX ft_profile (PROFILE);'


In [59]:
query_indx_5 = """SELECT *
    FROM discogs.artist
    WHERE MATCH(PROFILE) AGAINST('рок' IN NATURAL LANGUAGE MODE);"""

get_df(query_indx_5)

Время выполнения запроса: 0.1234 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,4239166,Крекерс,Krekers,Группа Крекерс (Krekers) из Санкт-Петербурга —...,Needs Vote
1,5107995,Гоген,,"Группа ""Гоген"" образовалась весной 1988 года, ...",Needs Vote
2,472264,Тамбурин,,"Группу создал певец, композитор, аранжировщик,...",Needs Vote
3,1620685,Гласовете,,"Групата „ГЛАСОВЕТЕ”, (или както е популярно то...",Needs Vote
4,2350272,Анатолий Азанов,,Photographer\r\rAnatoly Azanov was born on Jun...,Needs Vote
...,...,...,...,...,...
132,6900991,Vedic Social Empire,Ведическая социальная империя,Группа основана 21 декабря 2018 года Ампером А...,Needs Vote
133,6960465,Д.Н.О.,,Д.Н.О. (aka D.N.O. aka DNO aka ДНО) is an alte...,Needs Vote
134,7166614,TIR (4),True Industrial Romantic,"Группа TIR, в состав которой входят вокалист М...",Needs Vote
135,7231392,Trappist System Trio,,Прогрессив-рок трио TS3 образовано в мае 2017 ...,Needs Vote


In [47]:
query = text("""drop INDEX ft_profile ON `artist`""")
df = get_query_db(query)

Время выполнения запроса: 0.1002 секунд


-----------------------------